# Library Setup

In [1]:
!pip install --force-reinstall --no-deps --no-index /kaggle/input/transformers-422/transformers-4.24.0-py3-none-any.whl

Processing /kaggle/input/transformers-422/transformers-4.24.0-py3-none-any.whl
  Attempting uninstall: transformers
    Found existing installation: transformers 4.20.1
    Uninstalling transformers-4.20.1:
      Successfully uninstalled transformers-4.20.1


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle
import transformers
import os
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import glob
import os
import wandb
import random
import matplotlib.pyplot as plt
import numpy as np
from transformers import AdamW
from text_unidecode import unidecode
import torch
from transformers import get_cosine_schedule_with_warmup
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModel, AutoTokenizer, AutoConfig
import re
from torch.nn import Module
import torch.nn as nn
from collections import Counter, defaultdict
from tqdm import tqdm
import unicodedata
from copy import deepcopy
import sys
import gc
import codecs

In [3]:
transformers.__version__

'4.24.0'

In [4]:
# CFG1 = {
#     "model_name": "../input/bigbirdrobertalarge/bigbird-roberta-large",
#     "type": "Other Models",
#     "targets": ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions'],
#     "weights": "../input/downloading-ell-bigbird",
#     "max_length": 4096,
#     "seed": 42,
#     "folds": 4,
#     "lr": 2e-5, 
#     "batch_size": 1,
#     "num_warmup_steps": 0.0,
#     "grad_accum": 1,
#     "pooler": None,
#     "weight_decay": 0.3,
#     "grad_norm": 1000,
#     "dropout": 0.0,
#     "multisample": False,
#     "optimizer": "AdamW",
#     "scheduler": "linear",
# }
# CFG1["tokenizer"] = AutoTokenizer.from_pretrained(CFG1["model_name"])

In [5]:
# CFG2 = {
#     "model_name": "../input/debertav3base",
#     "type": "Other Models",
#     "targets": ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions'],
#     "weights": "../input/downloading-ell-debertav3base-notebooks",
#     "max_length": 512,
#     "seed": 42,
#     "folds": 4,
#     "lr": 2e-5, 
#     "batch_size": 1,
#     "num_warmup_steps": 0.0,
#     "grad_accum": 1,
#     "pooler": None,
#     "weight_decay": 0.3,
#     "grad_norm": 1000,
#     "dropout": 0.0,
#     "multisample": False,
#     "optimizer": "AdamW",
#     "scheduler": "linear",
# }
# CFG2["tokenizer"] = AutoTokenizer.from_pretrained(CFG2["model_name"])

In [6]:
# CFG3 = {
#     "model_name": "../input/deberta-v3-large/deberta-v3-large",
#     "type": "Other Models",
#     "targets": ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions'],
#     "weights": "../input/debertav3large-ell-download",
#     "max_length": 512,
#     "seed": 42,
#     "folds": 4,
#     "lr": 2e-5, 
#     "batch_size": 1,
#     "epochs": 20,
#     "num_warmup_steps": 0.0,
#     "patience": 6,
#     "grad_accum": 1,
#     "pooler": None,
#     "weight_decay": 0.3,
#     "grad_norm": 1000,
#     "dropout": 0.0,
#     "multisample": False,
#     "optimizer": "AdamW",
#     "scheduler": "linear",
# }
# CFG3["tokenizer"] = AutoTokenizer.from_pretrained(CFG3["model_name"])

In [7]:
# CFG4 = {
#     "model_name": "../input/allenailongformerbase4096/longformer",
#     "type": "Full Input",
#     "targets": ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions'],
#     "weights": "../input/downloading-ell-longformer",
#     "max_length": 4096,
#     "seed": 42,
#     "folds": 4,
#     "lr": 2e-5, 
#     "batch_size": 1,
#     "epochs": 32,
#     "num_warmup_steps": 0.0,
#     "patience": 6,
#     "grad_accum": 8,
#     "pooler": None,
#     "weight_decay": 0.3,
#     "grad_norm": 1000,
#     "dropout": 0.0,
#     "multisample": False,
#     "optimizer": "AdamW",
#     "scheduler": "linear",
# }
# CFG4["tokenizer"] = AutoTokenizer.from_pretrained(CFG4["model_name"])

In [8]:
# CFG5 = {
#     "model_name": "../input/deberta-v3-large/deberta-v3-large",
#     "type": "Other Models",
#     "targets": ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions'],
#     "weights": "../input/ell-pseudo-download",
#     "max_length": 512,
#     "seed": 42,
#     "folds": 4,
#     "lr": 2e-5, 
#     "batch_size": 1,
#     "epochs": 20,
#     "num_warmup_steps": 0.0,
#     "patience": 6,
#     "grad_accum": 1,
#     "pooler": None,
#     "weight_decay": 0.3,
#     "grad_norm": 1000,
#     "dropout": 0.0,
#     "multisample": False,
#     "optimizer": "AdamW",
#     "scheduler": "linear",
# }
# CFG5["tokenizer"] = AutoTokenizer.from_pretrained(CFG5["model_name"])

In [9]:
# CFG6 = {
#     "model_name": "../input/debertav3base",
#     "type": "Attention Regression Head",
#     "targets": ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions'],
#     "weights": "../input/downloading-attention-regression-head-debertav3b",
#     "max_length": 512,
#     "seed": 42,
#     "folds": 4,
#     "lr": 2e-5, 
#     "batch_size": 16,
#     "epochs": 20,
#     "num_warmup_steps": 0.0,
#     "patience": 6,
#     "grad_accum": 1,
#     "pooler": "attention",
#     "layer_start": 1,
#     "weight_decay": 0.3,
#     "grad_norm": 1000,
#     "hidden_dim": 128,
#     "dropout": 0.0,
#     "multisample": False,
#     "optimizer": "AdamW",
#     "scheduler": "linear",
# }
# CFG6["tokenizer"] = AutoTokenizer.from_pretrained(CFG6["model_name"])

In [10]:
# CFG7 = {
#     "model_name": "../input/debertav3base",
#     "type": "Weighted Regression Head",
#     "targets": ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions'],
#     "weights": "../input/downloading-weighted-head-debertav3base",
#     "max_length": 512,
#     "seed": 42,
#     "folds": 4,
#     "lr": 2e-5, 
#     "batch_size": 16,
#     "epochs": 20,
#     "num_warmup_steps": 0.0,
#     "patience": 6,
#     "grad_accum": 1,
#     "pooler": "weighted",
#     "layer_start": 9,
#     "weight_decay": 0.3,
#     "grad_norm": 1000,
#     "dropout": 0.0,
#     "multisample": False,
#     "optimizer": "AdamW",
#     "scheduler": "linear",
# }
# CFG7["tokenizer"] = AutoTokenizer.from_pretrained(CFG7["model_name"])

In [11]:
# CFG8 = {
#     "model_name": "../input/debertav3base",
#     "type": "Attention Regression Head + Multisample Dropout",
#     "targets": ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions'],
#     "weights": "../input/downloading-attention-multisample-debertav3base",
#     "max_length": 512,
#     "seed": 42,
#     "folds": 4,
#     "lr": 2e-5, 
#     "batch_size": 16,
#     "epochs": 20,
#     "num_warmup_steps": 0.0,
#     "patience": 6,
#     "grad_accum": 1,
#     "pooler": "attention",
#     "layer_start": 1,
#     "weight_decay": 0.3,
#     "grad_norm": 1000,
#     "hidden_dim": 128,
#     "dropout": 0.3,
#     "multisample": True,
#     "optimizer": "AdamW",
#     "scheduler": "linear",
# }
# CFG8["tokenizer"] = AutoTokenizer.from_pretrained(CFG8["model_name"])

In [12]:
# CFG9 = {
#     "model_name": "../input/deberta-v3-large/deberta-v3-large",
#     "type": "DS All",
#     "targets": ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions'],
#     "weights": "../input/downloading-debertav3large-l1-ds-all", 
#     "max_length": 512,
#     "seed": 42,
#     "folds": 4,
#     "lr": 2e-5, 
#     "batch_size": 4,
#     "epochs": 20,
#     "num_warmup_steps": 0.0,
#     "patience": 6,
#     "grad_accum": 4,
#     "pooler": "DS All",
#     "layer_start": 9,
#     "weight_decay": 0.3,
#     "grad_norm": 1000,
#     "aux_weight": 0.5,
#     "dropout": 0.1,
#     "oof_path": "deberta-v3-large-L1.csv", 
#     "multisample": False,
#     "optimizer": "AdamW",
#     "scheduler": "linear",
# }
# CFG9["tokenizer"] = AutoTokenizer.from_pretrained(CFG9["model_name"])

In [13]:
CFG14 = {
    "model_name": "../input/deberta-v3-large/deberta-v3-large",
    "type": "DS Last",
    "targets": ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions'],
    "weights": "/kaggle/input/downloading-debertav3large-l1-ds-last",
    "max_length": 512,
    "seed": 42,
    "folds": 4,
    "lr": 2e-5, 
    "batch_size": 4,
    "epochs": 20,
    "num_warmup_steps": 0.0,
    "patience": 6,
    "grad_accum": 4,
    "pooler": "DS Last",
    "layer_start": 9,
    "weight_decay": 0.3,
    "grad_norm": 1000,
    "aux_weight": 0.5,
    "dropout": 0.1,
    "multisample": False,
    "oof_path": "deberta-v3-large-L1-DS-Last.csv", 
    "optimizer": "AdamW",
    "scheduler": "linear",
}
CFG14["tokenizer"] = AutoTokenizer.from_pretrained(CFG14["model_name"])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/opt/conda/lib/python3.7/site-packages/transformers/convert_slow_tokenizer.py:447: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [14]:
CFG13 = {
    "model_name": "/kaggle/input/ernie20largeen/nghuyong/ernie-2.0-large-en",
    "type": "Other Models",
    "targets": ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions'],
    "weights": "/kaggle/input/downloading-ernie-2-0-large",
    "max_length": 512,
    "seed": 42,
    "folds": 4,
    "lr": 2e-5, 
    "batch_size": 16,
    "epochs": 20,
    "num_warmup_steps": 0.0,
    "patience": 6,
    "grad_accum": 1,
    "pooler": None,
    "weight_decay": 0.3,
    "grad_norm": 1000,
    "multisample": False,
    "dropout": 0.0,
    "optimizer": "AdamW",
    "scheduler": "linear",
}
CFG13["tokenizer"] = AutoTokenizer.from_pretrained(CFG13["model_name"])

# Model Definition

In [15]:
# CFGS = [CFG1, CFG2, CFG3, CFG5, CFG6, CFG7, CFG8]
CFGS = [CFG13]

In [16]:
class WeightedLayerPooling(torch.nn.Module):
    def __init__(self, num_hidden_layers, layer_start: int = 4, layer_weights = None):
        super(WeightedLayerPooling, self).__init__()
        self.layer_start = layer_start
        self.num_hidden_layers = num_hidden_layers
        self.layer_weights = layer_weights if layer_weights is not None \
            else nn.Parameter(
                torch.tensor([1] * (num_hidden_layers+1 - layer_start), dtype=torch.float)
            )

    def forward(self, all_hidden_states):
        all_layer_embedding = all_hidden_states[self.layer_start:, :, :, :]
        weight_factor = self.layer_weights.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).expand(all_layer_embedding.size())
        weighted_average = (weight_factor*all_layer_embedding).sum(dim=0) / self.layer_weights.sum()
        return weighted_average
    

In [17]:
class AttentionPooling(nn.Module):
    def __init__(self, in_features, hidden_dim):
        super().__init__()
        self.in_features = in_features
        self.W = nn.Linear(in_features, hidden_dim)
        self.V = nn.Linear(hidden_dim, 1)
        self.out_features = hidden_dim

    def forward(self, features):
        att = torch.tanh(self.W(features))
        score = self.V(att)
        attention_weights = torch.softmax(score, dim=0)
        context_vector = attention_weights * features
        context_vector = torch.sum(context_vector, dim=0)

        return context_vector

In [18]:
#OBVIOUSLY, CHANGE THIS AS YOU NEED. USE SELF.LOG FOR ALL IMPORTANT METRICS
class Model(nn.Module):
    def __init__(self, config, vocab_length, data_loader_len):
        super(Model, self).__init__()
        self.config = config
        self.vocab_length = vocab_length
        self.base_model = AutoModel.from_pretrained(self.config['model_name'], output_hidden_states = True)  
        self.base_model.resize_token_embeddings(vocab_length)
        self.dropout = torch.nn.Dropout(p=CFG["dropout"])
        self.order = torch.LongTensor([5, 0, 1, 2, 3, 4]).cuda()
        
        if self.config["pooler"] == "weighted":
            self.pooler = WeightedLayerPooling(self.base_model.config.num_hidden_layers, layer_start = self.config["layer_start"])  
            self._init_weights(self.pooler.layer_weights)
            
        elif self.config["pooler"] == "attention":
            self.pooler = AttentionPooling(self.base_model.config.hidden_size, config["hidden_dim"])

        if self.config["multisample"]:
            self.dropout1 = nn.Dropout(0.1)
            self.dropout2 = nn.Dropout(0.2)
            self.dropout3 = nn.Dropout(0.3)
            self.dropout4 = nn.Dropout(0.4)
            self.dropout5 = nn.Dropout(0.5)
            
        self.dropout = nn.Dropout(self.config["dropout"])
        self.fc = nn.Linear(self.base_model.config.hidden_size, 6)
        self._init_weights(self.fc)
        
        if config["pooler"] == "DS All" or config["pooler"] == "DS MaxPool":
            self.fcs = nn.ModuleList([])
            for _ in range(6):
                layer = nn.Linear(self.base_model.config.hidden_size, 1)
                self._init_weights(layer)
                self.fcs.append(layer)
        else:
            self.fcs = nn.ModuleList([])
            for _ in range(7):
                layer = nn.Linear(self.base_model.config.hidden_size, 1)
                self._init_weights(layer)
                self.fcs.append(layer)
        
        self.data_loader_len = data_loader_len

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.base_model.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.base_model.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)

    def feature(self, inputs):

        if self.config["pooler"] == "weighted":
            input_ids, attention_mask = inputs["input_ids"], inputs["attention_mask"]
        
            x = self.base_model(input_ids = input_ids, attention_mask = attention_mask)["hidden_states"]

            x = torch.stack(x)
            cls_embeddings = self.pooler(x)[:, 0]

            return cls_embeddings
        
        
        elif self.config["pooler"] == "DS Last":
            input_ids, attention_mask = inputs["input_ids"], inputs["attention_mask"]
        
            x = self.base_model(input_ids = input_ids, attention_mask = attention_mask)["hidden_states"]

            x = torch.stack(x)

            # Last 6 layers
            return x[-7:, :, :, :]
        
        elif self.config["pooler"] == "DS All":
            input_ids, attention_mask = inputs["input_ids"], inputs["attention_mask"]
        
            x = self.base_model(input_ids = input_ids, attention_mask = attention_mask)["hidden_states"]

            x = torch.stack(x)

            # Last 6 layers
            return x[-6:, :, :, :]
        
        elif self.config["pooler"] == "DS MaxPool":
            input_ids, attention_mask = inputs["input_ids"], inputs["attention_mask"]
        
            x = self.base_model(input_ids = input_ids, attention_mask = attention_mask)["hidden_states"]

            x = torch.stack(x)

            # Last 6 layers
            return x[-6:, :, :, :]
            
        else:
            input_ids, attention_mask = inputs["input_ids"], inputs["attention_mask"]
        
            x = self.base_model(input_ids = input_ids, attention_mask = attention_mask)["last_hidden_state"]

            return x[:, 0, :]

            
    def forward(self, inputs):
        
        features = self.feature(inputs)
        
        if self.config["multisample"]:
            logits1 = self.fc(self.dropout1(features))
            logits2 = self.fc(self.dropout2(features))
            logits3 = self.fc(self.dropout3(features))
            logits4 = self.fc(self.dropout4(features))
            logits5 = self.fc(self.dropout5(features))

            logits = (logits1 + logits2 + logits3 + logits4 + logits5) / 5
            
            return logits

            
        if self.config["pooler"] == "DS Last":

            outputs = []

            for layer_num, layer in enumerate(features):

                if layer_num == (len(features) - 1):
                    pred = self.fc(self.dropout(layer[:, 0, :]))
                    break
            
                outputs.append(self.fcs[layer_num](self.dropout(layer[:, 0, :])))

            outputs = torch.stack(outputs)
            
            return pred
        
        elif self.config["pooler"] == "DS MaxPool":

            outputs = []

            layers = []

            for layer_num, layer in enumerate(features):
                
                layers.append(self.dropout(layer[:, 0, :]))
                
                outputs.append(self.fcs[layer_num](layers[-1]))

            outputs = torch.stack(outputs)

            layers = torch.stack(layers)

            final_cls = torch.max(layers, dim = 0)[0]

            pred = checkpoint(self.fc, final_cls)
            
            return pred
        
        elif self.config["pooler"] == "DS All":

            outputs = []

            layers = []

            for layer_num, layer in enumerate(features):
                
                layers.append(self.dropout(layer[:, 0, :]))
                
                outputs.append(self.fcs[layer_num](layers[-1]))
            
            outputs = torch.stack(outputs)

            return torch.index_select(outputs.squeeze(-1).transpose(0,1), 1, self.order)
        
        else:
            logits = self.fc(features)
            
            return logits
            

# Dataset Classes

In [19]:
class TestData(Dataset):
    def __init__(self, df, config, base_path, special_tokens = None):
        self.df = df
        self.esc_chars = ['\"', "\\", "\n", "\r", "\t", "\b", "\f", "\v", ":)", ";)", ":(", "uwu", "owo", "xd", ":3", ":-)", ":D", ">:(", "\xa0", "\x92", "\x93", "\x91", "\x94", "\x97", "x\B4", "\x96", "\x82", "\x84"]
        self.df["full_text"] = self.df["full_text"].apply(lambda text: self.resolve_encodings_and_normalize(text))

        codecs.register_error("replace_encoding_with_utf8", self.replace_encoding_with_utf8)
        codecs.register_error("replace_decoding_with_cp1252", self.replace_decoding_with_cp1252)

    def replace_encoding_with_utf8(self, error):
        return error.object[error.start : error.end].encode("utf-8"), error.end


    def replace_decoding_with_cp1252(self, error):
        return error.object[error.start : error.end].decode("cp1252"), error.end


    def resolve_encodings_and_normalize(self, text: str) -> str:
        text = (
            text.encode("raw_unicode_escape")
            .decode("utf-8", errors="replace_decoding_with_cp1252")
            .encode("cp1252", errors="replace_encoding_with_utf8")
            .decode("utf-8", errors="replace_decoding_with_cp1252")
        )
        
        text = unidecode(text)
        
        return self.remove_esc_chars(text)

    def remove_esc_chars(self, text):
        txt = deepcopy(text)
        for char in self.esc_chars:
            if char == '\"':
                txt = txt.replace(char, '"')
            elif char == "\x92" or char == "\x91" or char == "\xB4":
                txt = txt.replace(char, "'")
            elif char == "\0x93" or char == "\0x94":
                txt = txt.replace(char, '"')
            elif char == "\0x97" or char == "\0x96":
                txt = txt.replace(char, '-')
            elif char == "\0x82" or char == "\0x84":
                txt = txt.replace(char, ',')
            else:
                txt = txt.replace(char, ' ')
        return txt

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        return self.df.iloc[idx]["full_text"]

In [20]:
def construct_collate_fn(config):
    def collate_dynamic_padding(batch):
        # Dynamic Padding tokenization
        sentences = config["tokenizer"](batch, padding=True, max_length = config["max_length"], truncation = True, return_token_type_ids = False, return_tensors="pt")
        return sentences
    
    return collate_dynamic_padding

In [21]:
#CHANGE AS NEEDED. MOST OF THE TIME, PYTORCH'S DEFAULT COLLATOR IS ENOUGH.
class DataModule():
    def __init__(self, config, test, collate_fn):
        self.config = config
        self.test = test
        self.collate_fn = collate_fn

    def test_dataloader(self):
        test_loader = DataLoader(self.test, batch_size = self.config["batch_size"], collate_fn = self.collate_fn)      
        return test_loader

# Inferencing

In [22]:
def predict(model, loader):
    outputs = []
    device = torch.device('cuda')
    model = model.to(device).eval()
    with torch.no_grad():
        for inputs in tqdm(loader):
            for key, value in inputs.items():
                inputs[key] = value.to(device)
            predictions = model(inputs)
            outputs.append(predictions)
    return torch.cat(tuple(outputs))

In [23]:
def get_predictions(loader, num_preds, path, config):
    outputs = []
    for fold in range(num_preds):

        model = Model(config, len(config["tokenizer"]), len(loader))
        checkpoint = torch.load(f"{path}/fold-{fold}.pt", map_location=torch.device('cpu'))
        model.load_state_dict(checkpoint['model_state_dict'], strict=False)
        outputs.append(predict(model, loader))
        
        del model, checkpoint; gc.collect()
        torch.cuda.empty_cache()
    return torch.stack(tuple(outputs))

In [24]:
all_preds = []
for CFG in CFGS:
    test = TestData(pd.read_csv("../input/feedback-prize-english-language-learning/test.csv"), CFG, "./")

    dataset = DataModule(CFG, test, construct_collate_fn(CFG))

    test_loader = dataset.test_dataloader()
    
    outputs = get_predictions(test_loader, num_preds = 4, path=CFG["weights"], config = CFG)
    
    all_preds.append(outputs)

100%|██████████| 1/1 [00:00<00:00, 30.07it/s]


In [25]:
all_preds[0].shape

torch.Size([4, 3, 6])

In [26]:
all_preds

[tensor([[[2.8320, 2.6279, 2.9077, 2.9311, 2.6271, 2.7072],
          [2.7152, 2.4567, 2.6243, 2.5590, 2.0677, 2.6769],
          [3.6321, 3.3606, 3.5519, 3.6467, 3.5881, 3.5491]],
 
         [[2.7915, 2.5918, 3.0143, 2.9139, 2.6483, 2.6162],
          [2.7407, 2.4680, 2.7743, 2.5778, 2.4082, 2.7162],
          [3.6322, 3.3006, 3.4676, 3.5488, 3.2469, 3.3507]],
 
         [[2.6924, 2.7878, 3.1450, 3.0423, 2.6908, 2.6850],
          [2.7947, 2.7697, 2.8286, 2.6390, 2.2946, 2.8212],
          [3.6556, 3.5074, 3.6025, 3.6201, 3.3463, 3.3978]],
 
         [[2.9577, 2.9801, 3.1688, 3.1578, 2.7648, 2.8780],
          [2.6192, 2.5910, 2.7561, 2.4139, 2.1595, 2.7531],
          [4.0686, 3.8909, 3.9064, 4.1088, 3.7099, 3.9025]]], device='cuda:0')]

In [27]:
preds = []
for num, pred in enumerate(all_preds):
    preds.append(torch.mean(pred, dim = 0))

In [28]:
final_preds = torch.mean(torch.stack(preds), dim = 0).cpu().numpy()

In [29]:
final_preds

array([[2.8184164, 2.7469065, 3.0589442, 3.011283 , 2.6827464, 2.7216218],
       [2.7174506, 2.5713854, 2.7458014, 2.5474372, 2.2324948, 2.7418466],
       [3.7471223, 3.5148609, 3.6321054, 3.7310827, 3.4727864, 3.5500383]],
      dtype=float32)

In [30]:
# ONLY FOR ONE CONFIGURATION
# final_preds = np.vstack(all_preds)
# final_preds = torch.stack(all_preds)
# final_preds = torch.mean(torch.squeeze(torch.stack(all_preds)), dim = 0).cpu().numpy()

# ONLY FOR MULTIPLE CONFIGURATIONS
# final_preds = torch.mean(torch.mean(torch.squeeze(torch.stack(all_preds)), dim = 0), dim = 0).cpu().numpy()

In [31]:
final_preds.shape

(3, 6)

In [32]:
final_preds

array([[2.8184164, 2.7469065, 3.0589442, 3.011283 , 2.6827464, 2.7216218],
       [2.7174506, 2.5713854, 2.7458014, 2.5474372, 2.2324948, 2.7418466],
       [3.7471223, 3.5148609, 3.6321054, 3.7310827, 3.4727864, 3.5500383]],
      dtype=float32)

In [33]:
t = pd.read_csv("../input/feedback-prize-english-language-learning/test.csv")
submission = pd.DataFrame({"text_id": t.text_id})
del t

In [34]:
submission["cohesion"], submission["syntax"], submission["vocabulary"], submission["phraseology"], submission["grammar"], submission["conventions"] = final_preds[:,0], final_preds[:,1], final_preds[:,2], final_preds[:,3], final_preds[:,4], final_preds[:,5]

In [35]:
submission.to_csv("submission.csv", index=False)

In [36]:
submission

,text_id,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0000C359D63E,2.818416,2.746907,3.058944,3.011283,2.682746,2.721622
1,000BAD50D026,2.717451,2.571385,2.745801,2.547437,2.232495,2.741847
2,00367BB2546B,3.747122,3.514861,3.632105,3.731083,3.472786,3.550038
